In [207]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC  
import pandas as pd
from geopy.geocoders import Nominatim
import time

In [208]:
def accept_cookies_if_asked(driver):
    try:
        driver.find_element(By.ID, "onetrust-accept-btn-handler").click()
    except:
        pass

In [209]:
class Data:
    def __init__(self):
        self.ids = []
        self.prices = []
        self.address = []

    def add_page_data(self, driver):
        properties = driver.find_elements(By.CLASS_NAME, "l-searchResult")
        for property in properties:
            self.ids.append(property.find_element(By.CLASS_NAME, "propertyCard-anchor").get_attribute('id'))
            self.prices.append(property.find_element(By.CLASS_NAME, "propertyCard-priceValue").text)
            self.address.append(property.find_element(By.CLASS_NAME, "propertyCard-address").get_attribute('title'))

    def transform_prices(self):
        self.prices = [int(price[1:-4].replace(',','')) for price in self.prices]

    def make_df(self):
        return pd.DataFrame({'id': self.ids, 'price': self.prices, 'address': self.address})


In [138]:
def get_page_data(driver, ids, prices, address):
    properties = driver.find_elements(By.CLASS_NAME, "l-searchResult")
    for property in properties:
        ids.append(property.find_element(By.CLASS_NAME, "propertyCard-anchor").get_attribute('id'))
        prices.append(property.find_element(By.CLASS_NAME, "propertyCard-priceValue").text)
        address.append(property.find_element(By.CLASS_NAME, "propertyCard-address").get_attribute('title'))

    return pd.DataFrame({'id': ids, 'price': prices, 'address': address})


In [210]:

url = r'''https://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=REGION%5E92829&maxBedrooms=2&minBedrooms=2&propertyTypes=&includeLetAgreed=false&mustHave=&dontShow=&furnishTypes=&keywords='''
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome()
driver.get(url)

time.sleep(2)
accept_cookies_if_asked(driver)

data = Data()
data.add_page_data(driver)
pages_scanned = 1

button = driver.find_element(By.XPATH, "//button[@title='Next page']")

while button.is_enabled():
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable(button))
    button.click()
    accept_cookies_if_asked(driver)
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'propertyCard-anchor')))
    data.add_page_data(driver)
    pages_scanned += 1
    button = driver.find_element(By.XPATH, "//button[@title='Next page']")

driver.close()


In [211]:
pages_scanned

42

In [214]:
data.transform_prices()

In [216]:
df = data.make_df()

In [217]:
df

,id,price,address
0,prop123249455,3380,"Wandsworth Road, London, SW8 2FW, UK"
1,prop137578016,1850,"Carmichael Court, Barnes, SW13"
2,prop137206811,2250,"West Elms Studios, Stewarts Road Battersea SW8"
3,prop137047472,2275,"St Lukes Court, Battersea, London"
4,prop111890363,2097,"Almanac House, 180 East Hill, Wandsworth, Lond..."
...,...,...,...
1045,prop135954134,4983,"Carnation Way, Thames City, SW8"
1046,prop134914124,4983,"Carnation Way,, Thames City, SW8"
1047,prop135617696,5200,"Carnation Way, Thames City, SW8"
1048,prop135948131,3250,"Lombard Road, Battersea, London, SW11"


In [181]:
df = pd.DataFrame({'id': data.ids, 'price': data.prices, 'address': data.address})

In [182]:
df['price'] = [int(price[1:-4].replace(',','')) for price in df.price]

In [183]:
df.sort_values('price')

,id,price,address
1001,prop136086893,1000,"Gilbert Close, Morden Road, London, SW19"
42,prop137528039,1500,"Flat 3, ArragonBishops Park RoadNorburyLondon"
49,prop137525111,1534,"Windsor Avenue, London, SW19"
716,prop136924028,1550,"Cotton Close, Mitcham, Surrey, CR4"
710,prop136936121,1600,"Imperial Gardens, Mitcham, Surrey, CR4"
...,...,...,...
5,prop137518448,7150,"Centurion Building, Chelsea Bridge Wharf, London"
969,prop135053315,7583,"The Tower, 1 St George Wharf, Vauxhall"
920,prop136384520,9000,"Anhalt Road, Battersea"
1017,prop134006660,9000,"Anhalt Road, Battersea"
